In [8]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)

# Chemin vers le fichier JSONL
file_path = '/home/carolus/Documents/school/green_ia/data/03_openfoodfacts_sample.jsonl'

# Lire le fichier JSONL en utilisant la méthode read_json de pandas avec l'option lines=True
df = pd.read_json(file_path, lines=True)

# Afficher le DataFrame
print(df)


               pnns_groups_1  \
0                    unknown   
1    Milk and dairy products   
2                    unknown   
3                    unknown   
4                    unknown   
..                       ...   
995           Fat and sauces   
996                  unknown   
997                  unknown   
998                  unknown   
999             Salty snacks   

                                         ecoscore_data  \
0    {'adjustments': {'origins_of_ingredients': {'e...   
1    {'grade': 'c', 'previous_data': {'agribalyse':...   
2    {'status': 'unknown', 'missing': {'categories'...   
3    {'missing_key_data': 1, 'missing_agribalyse_ma...   
4    {'adjustments': {'origins_of_ingredients': {'t...   
..                                                 ...   
995  {'adjustments': {'threatened_species': {'warni...   
996  {'adjustments': {'production_system': {'value'...   
997  {'adjustments': {'production_system': {'value'...   
998  {'missing': {'origins': 1, 'in

In [21]:
df.head(1)

,groups,ecoscore_data,ingredients,packaging,name,food_groups,ecoscore_groups,categories,ecoscore_note,labels,countries
0,unknown,{'adjustments': {'origins_of_ingredients': {'e...,"[en:maltodextrin, en:instant-noodles, en:wheat...",NaN,minestrone soup with noodles,[],[unknown],NaN,NaN,"[en:sustainable, en:sustainable-palm-oil]",en:Australia


In [26]:
# main preprocessing

df.rename(columns={'pnns_groups_1': 'groups'}, inplace=True)
df.rename(columns={'ingredients_tags': 'ingredients'}, inplace=True)
df.rename(columns={'product_name': 'name'}, inplace=True)
df.rename(columns={'food_groups_tags': 'food_groups'}, inplace=True)
df.rename(columns={'ecoscore_tags': 'ecoscore_groups'}, inplace=True)
df.rename(columns={'categories_tags': 'categories'}, inplace=True)
df.rename(columns={'ecoscore_score': 'ecoscore_note'}, inplace=True)
df.rename(columns={'labels_tags': 'labels'}, inplace=True)

In [27]:
df.head(30)

,groups,ecoscore_data,ingredients,packaging,name,food_groups,ecoscore_groups,categories,ecoscore_note,labels,countries
0,NaN,{'adjustments': {'origins_of_ingredients': {'e...,"[en:maltodextrin, en:instant-noodles, en:wheat...",NaN,minestrone soup with noodles,[],[unknown],NaN,NaN,"[en:sustainable, en:sustainable-palm-oil]",en:Australia
1,Milk and dairy products,"{'grade': 'c', 'previous_data': {'agribalyse':...",NaN,,Ricotta,"[en:milk-and-dairy-products, en:cheese]",[c],"[en:dairies, en:fermented-foods, en:fermented-...",50.0,[],Italy
2,NaN,"{'status': 'unknown', 'missing': {'categories'...",NaN,NaN,Roggenmischbrot,[],[unknown],NaN,NaN,NaN,en:de
3,NaN,"{'missing_key_data': 1, 'missing_agribalyse_ma...",NaN,NaN,Galette frangipane,[],[unknown],NaN,NaN,NaN,en:fr
4,NaN,{'adjustments': {'origins_of_ingredients': {'t...,"[de:vollmilchpulver-mit-löslichem-kaffee, en:w...",Kaffekapseln,Café au Lait,[],[e],"[en:plant-based-foods-and-beverages, en:bevera...",-20.0,[],"Germany, en:france"
5,NaN,"{'missing': {'origins': 1, 'labels': 1, 'categ...",NaN,NaN,chocolate hazelnut butter,[],[unknown],NaN,NaN,NaN,France
6,NaN,"{'missing': {'origins': 1, 'categories': 1, 'l...",NaN,NaN,Tomates ronde,[],[unknown],NaN,NaN,NaN,en:France
7,Fat and sauces,{'agribalyse': {'warning': 'missing_agribalyse...,"[en:water, en:vinegar, en:arbol-chili-powder, ...",NaN,"Rancho gordo, rio fuego very hot sauce, hot","[en:fats-and-sauces, en:dressings-and-sauces]",[unknown],"[en:condiments, en:sauces, en:hot-sauces, en:g...",NaN,[],United States
8,NaN,{'adjustments': {'threatened_species': {'warni...,NaN,NaN,Alkaline purified water with electrolytes adde...,[],[unknown],NaN,NaN,NaN,en:us
9,NaN,{'adjustments': {'packaging': {'non_recyclable...,NaN,NaN,NaN,[],[unknown],NaN,NaN,NaN,en:Albania


In [12]:
nan_counts = df.isna().sum()

# Compter les listes vides
empty_list_counts = df.map(lambda x: x == []).sum()

# Afficher les résultats
print("nombre de NaN ou None par colonne :")
print(nan_counts)

nombre de NaN ou None par colonne :
groups               4
ecoscore_data        4
ingredients        724
packaging          788
name                43
food_groups          4
ecoscore_groups      4
categories         531
ecoscore_note      752
labels             589
countries            6
dtype: int64


In [13]:
print("nombre de listes vides par colonne :")
print(empty_list_counts)

nombre de listes vides par colonne :
groups               0
ecoscore_data        0
ingredients          0
packaging            0
name                 0
food_groups        644
ecoscore_groups      0
categories          32
ecoscore_note        0
labels             121
countries            0
dtype: int64


In [16]:
df['labels'].head(50)

0             [en:sustainable, en:sustainable-palm-oil]
1                                                    []
2                                                  None
3                                                  None
4                                                    []
5                                                  None
6                                                  None
7                                                    []
8                                                  None
9                                                  None
10                                                 None
11                                                 None
12                                                 None
13    [en:not-advised-for-specific-people, en:not-ad...
14         [en:organic, en:no-gmos, en:non-gmo-project]
15    [en:not-advised-for-specific-people, en:green-...
16                                                 None
17                                              

In [ ]:
df['ingredients_tags'].head(50)

0                                                  None
1                                                  None
2                                                  None
3                                                  None
4                                                  None
5                                                  None
6                                                  None
7     [en:protein-blend, en:natural-prebiotic-solubl...
8                                                  None
9                                                  None
10    [en:kale, en:vegetable, en:brassica, en:cabbag...
11                                                 None
12                                                 None
13                                                 None
14                                                 None
15    [en:chickpea, en:vegetable, en:legume, en:puls...
16    [en:pineapple, en:fruit, en:water, en:sugar, e...
17    [en:skimmed-milk, en:dairy, en:milk, en:wa